In [13]:
from cached_property import cached_property

class C:
    
    def __init__(self, **kw):
        self.__dict__.update(kw)
        
    a = D()

In [15]:
obj = C(a=7)

In [16]:
obj.a

7

In [19]:
C.__dict__['a'].__get__(obj)

3

In [11]:
cached_property??

Init signature: cached_property(func)
Source:        
class cached_property(object):
    """
    A property that is only computed once per instance and then replaces itself
    with an ordinary attribute. Deleting the attribute resets the property.
    Source: https://github.com/bottlepy/bottle/commit/fa7733e075da0d790d809aa3d2f53071897e6f76
    """  # noqa

    def __init__(self, func):
        self.__doc__ = getattr(func, "__doc__")
        self.func = func

    def __get__(self, obj, cls):
        if obj is None:
            return self

        if asyncio and asyncio.iscoroutinefunction(self.func):
            return self._wrap_in_coroutine(obj)

        value = obj.__dict__[self.func.__name__] = self.func(obj)
        return value

    def _wrap_in_coroutine(self, obj):

        @asyncio.coroutine
        def wrapper():
            future = asyncio.ensure_future(self.func(obj))
            obj.__dict__[self.func.__name__] = future
            return future

        return wrapper(

In [1]:
from dataclasses import dataclass

@dataclass()
class DC:
    a: object

In [2]:
DC()

TypeError: __init__() missing 1 required positional argument: 'a'

In [3]:
dataclass??

Signature:
dataclass(
    ['_cls=None', '*', 'init=True', 'repr=True', 'eq=True', 'order=False', 'unsafe_hash=False', 'frozen=False'],
)
Source:   
def dataclass(_cls=None, *, init=True, repr=True, eq=True, order=False,
              unsafe_hash=False, frozen=False):
    """Returns the same class as was passed in, with dunder methods
    added based on the fields defined in the class.

    Examines PEP 526 __annotations__ to determine fields.

    If init is true, an __init__() method is added to the class. If
    repr is true, a __repr__() method is added. If order is true, rich
    comparison dunder methods are added. If unsafe_hash is true, a
    __hash__() method function is added. If frozen is true, fields may
    not be assigned to after instance creation.
    """

    def wrap(cls):
        return _process_class(cls, init, repr, eq, order, unsafe_hash, frozen)

    # See if we're being called as @dataclass or @dataclass().
    if _cls is None:
        # We're called with parens.